# População por subprefeitura

Para calcularmos diversos dados de maneira proporcional à população por subprefeitura, precisamos inicialmente obter a população de cada uma delas. Para isso, utilizamos os dados do Censo Demográfico de 2022, agregados por distritos, e reagregamos esses dados para o nível de subprefeitura.

In [ ]:
import pandas as pd
import geopandas as gpd
from os import path, makedirs


from core.downloads.geosampa import get_capabilities, get_features

# Obtendo os dados

## Obtendo os dados do Censo Demográfico de 2022

In [ ]:
url_pop = 'https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/Agregados_por_Distrito_csv/Agregados_por_distritos_demografia_BR.zip'

df_pop = pd.read_csv(
    url_pop,
    encoding='latin1',
    sep=';',
    dtype=str
)

df_pop.head()

Primeiro, vamos filtrar os dados para o município de São Paulo.

In [ ]:
df_pop = df_pop[df_pop['CD_DIST'].str.startswith('3550308')]
df_pop

Agora, vamos selecionar apenas as colunas de interesse, que são as colunas de população por faixa etária. Essas variáveis são as seguintes:

| Tema | Variável | Descrição |
|------|----------|-----------|
| Demografia | V01031 | 0 a 4 anos |
| Demografia | V01032 | 5 a 9 anos |
| Demografia | V01033 | 10 a 14 anos |
| Demografia | V01034 | 15 a 19 anos |
| Demografia | V01035 | 20 a 24 anos |
| Demografia | V01036 | 25 a 29 anos |
| Demografia | V01037 | 30 a 39 anos |
| Demografia | V01038 | 40 a 49 anos |
| Demografia | V01039 | 50 a 59 anos |
| Demografia | V01040 | 60 a 69 anos |
| Demografia | V01041 | 70 anos ou mais |

In [ ]:
cols_needed = ['CD_DIST', 'NM_DIST', 'V01031', 'V01032', 'V01033',
               'V01034', 'V01035', 'V01036', 'V01037', 'V01038',
               'V01039', 'V01040', 'V01041',]

df_pop = df_pop[cols_needed]
df_pop

Por último, vamos convertes as colunas de população para o tipo inteiro.

In [ ]:
df_pop = df_pop.astype({c: 'int' for c in cols_needed[2:]})
df_pop

Como não houve nenhum erro, sabemos que todos os valores de todas as colunas eram numéricos.

## Distritos e Subprefeituras

Também precisaremos saber a qual subprefeitura cada distrito pertence. Para isso, vamos utilizar os dados disponibilizados pelo Geosampa.

In [ ]:
get_capabilities('distrito')

In [ ]:
gdf_distritos = get_features('geoportal:distrito_municipal')
gdf_distritos.head()

In [ ]:
get_capabilities('subprefeitura')

In [ ]:
gdf_subs = get_features('geoportal:subprefeitura')
gdf_subs.head()

Vamos filtrar as colunas das tabelas de distritos e subprefeituras para obter apenas as colunas de interesse.

In [ ]:
dist_cols = ['id', 'cd_identificador_distrito',
             'cd_identificador_subprefeitura', 'cd_distrito_municipal',
             'nm_distrito_municipal', 'sg_distrito_municipal']

sub_cols = ['cd_subprefeitura', 'nm_subprefeitura']

df_distritos = gdf_distritos[dist_cols]
df_subs = gdf_subs[sub_cols]

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [ ]:
output_dir = path.join('dados')

if not path.exists(output_dir):
    makedirs(output_dir)

for c, df in [('distritos_original', df_distritos),
               ('subprefeituras_original', df_subs),
               ('censo_demografia_original', df_pop),
               ]:
    filename=path.join(output_dir, c)
    df.to_csv(f'{filename}.csv',
              sep=';',
              decimal=',',
              encoding='latin1',
              index=False
              )